<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Fiyat_Daralma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  openpyxl ta tqdm
!pip install tradingview-screener==2.5.0

import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from tqdm import tqdm

# Yüksek ve Düşük Daralan Fiyat Aralığı Hesaplama Fonksiyonu
def detect_consolidation_area(data, window=5, max_range_pct=0.01):
    recent_high = data['High'].rolling(window=window).max()
    recent_low = data['Low'].rolling(window=window).min()
    range_pct = (recent_high - recent_low) / data['Close']
    return range_pct < max_range_pct

# Küçük Gövdeli Mum Yapısı Kontrolü
def detect_small_body_candles(data, body_threshold_pct=0.005):
    body_size = abs(data['Close'] - data['Open']) / data['Close']
    return body_size < body_threshold_pct

# BIST'teki tüm hisse senetlerini otomatik çekme
def get_all_stocks():
    tv = TvDatafeed()
    stocks = get_all_symbols(market='turkey')
    stocks = [symbol.replace('BIST:', '') for symbol in stocks]
    return sorted(stocks)

# Kullanıcının birden fazla zaman dilimi seçmesini sağlayan fonksiyon
def get_intervals_choices():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen tarama yapılacak zaman dilim(ler)ini seçin (örn. 1,3,5):")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choices = input("Seçiminizi yapın (örneğin 1,3,5): ")
    selected_intervals = []

    for choice in choices.split(','):
        choice = choice.strip()
        if choice in intervals:
            selected_intervals.append(intervals[choice][1])  # Seçilen zaman dilimlerini ekler

    if not selected_intervals:
        print("Geçersiz seçim yapıldı. Varsayılan olarak günlük interval seçildi.")
        selected_intervals.append(Interval.in_daily)  # Varsayılan

    return selected_intervals

# Price Action'a Göre Sıkışma Taraması
def find_price_action_consolidation(stock_list, intervals):
    tv = TvDatafeed()
    results = []

    for stock in tqdm(stock_list, desc="Price Action Taraması"):
        for interval in intervals:
            try:
                data = tv.get_hist(symbol=stock, exchange='BIST', interval=interval, n_bars=100)

                if data is None or data.empty:
                    print(f"{stock} için {interval} zaman diliminde veri bulunamadı.")
                    continue

                data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

                # Fiyat Aralığı Sıkışması ve Küçük Gövdeli Mumları Tespit Et
                data['Consolidation_Area'] = detect_consolidation_area(data)
                data['Small_Body'] = detect_small_body_candles(data)

                # Sıkışma Bölgesi Koşulları Sağlayanlar
                if data['Consolidation_Area'].iloc[-1] and data['Small_Body'].iloc[-1]:
                    results.append([stock, data['Close'].iloc[-1], "Yatay Sıkışma Tespit Edildi", interval])
                    print(f"{stock} - Yatay sıkışma tespit edildi - Zaman Dilimi: {interval}.")

            except Exception as e:
                print(f"{stock} işlenirken hata: {e}")

    df_results = pd.DataFrame(results, columns=['Hisse Adı', 'Son Fiyat', 'Durum', 'Zaman Dilimi'])
    return df_results

# Kullanıcıdan zaman dilimlerini al
intervals = get_intervals_choices()

# Tüm hisseleri çek
stock_list = get_all_stocks()

# Price action analizi yap ve sonuçları göster
df_results = find_price_action_consolidation(stock_list, intervals)
print("Yatay Sıkışma Taraması Sonuçları:")
print(df_results)

# Excel dosyasına kaydet
excel_file = 'price_action_consolidation_results.xlsx'
df_results.to_excel(excel_file, index=False)
print(f"Yatay sıkışma taraması sonuçları '{excel_file}' dosyasına kaydedildi.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-mczryn0o
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-mczryn0o
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=404f6404056b5f54166f916777f2c6d61604b1c28a0638e01df7a0d4763280fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-asp68k_q/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=97063f7173c1516bdc02aa7bf0c61fb7c8b4f68a237ff4b59b90b23ece2cf309
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built tvdatafeed ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5

Price Action Taraması:  94%|█████████▍| 548/583 [05:07<00:19,  1.82it/s]

USDTR - Yatay sıkışma tespit edildi - Zaman Dilimi: Interval.in_daily.


Price Action Taraması: 100%|██████████| 583/583 [05:25<00:00,  1.79it/s]


Yatay Sıkışma Taraması Sonuçları:
  Hisse Adı  Son Fiyat                        Durum       Zaman Dilimi
0     USDTR     3324.0  Yatay Sıkışma Tespit Edildi  Interval.in_daily
Yatay sıkışma taraması sonuçları 'price_action_consolidation_results.xlsx' dosyasına kaydedildi.
